In [9]:
#linear regression
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load dataset
data = pd.read_csv('train_split_no_missing_data.csv')
data['date'] = pd.to_datetime(data['date'])

# Feature and target
target = "umsatz"
def prepare_features(df_subset):
    df_copy = df_subset.copy()
    df_copy['Monat'] = df_copy['date'].dt.month
    df_copy['Wochentag'] = df_copy['date'].dt.weekday
    X = df_copy.drop(columns=[target, 'date'])
    if 'Wettercode' in X.columns:
        X = pd.get_dummies(X, columns=['Wettercode']) #because it's categorical
    X = X.select_dtypes(include='number')
    return X

X_train = prepare_features(data)
y_train = data[target]

model = LinearRegression()
model.fit(X_train, y_train)

def add_predictions(df_subset, X_subset):
    df_out = df_subset.copy()
    df_out['Prediction'] = model.predict(X_subset)
    return df_out

train_out = add_predictions(data, X_train)

#save
train_out.to_csv("train_predictions_byLone.csv", index=False)

In [10]:
import pandas as pd

df = pd.read_csv("train_predictions_byLone.csv")

# delete unnecessary columns for kaggle
df = df.drop(columns=['date', 'Bewoelkung', 'Windgeschwindigkeit', 'KielerWoche', 'Temperatur', 'warengruppe', 'Wettercode', 'umsatz', 'school_holiday', 'public_holiday', 'Tag', 'Monat'])

#rename column for submission
df = df.rename(columns={'Prediction': 'umsatz'})
df.to_csv("train_predictions_byLone_final.csv", index=False)